notebook: https://github.com/some-labs-24/data-science/blob/master/python_notebooks/SoMe_NLP_Topic_Modeling.ipynb

In [29]:
# !pip install emoji --upgrade
# !pip install pandas-profiling==2.*
# !pip install plotly==4.*
# !pip install pyldavis
# !pip install gensim
# !pip install chart_studio
# !pip install --upgrade autopep8

# # !pip install -U pip setuptools wheel
# # !pip install -U spacy
# # !python -m spacy download fr_core_news_lg

In [2]:
# Required Libraries

#Base and Cleaning 
import json
import requests
import pandas as pd
import numpy as np
import emoji
import regex
import re
import string
from collections import Counter

In [3]:
#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 
import pyLDAvis.gensim

In [4]:
#Natural Language Processing (NLP)
import spacy
import gensim
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
#from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
#from wordcloud import STOPWORDS
#stopwords = set(STOPWORDS)

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/anaconda/envs/pfe-env/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [5]:
!pip install demoji

import demoji
demoji.download_codes()

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


... OK (Got response in 0.59 seconds)
Writing emoji data to /home/azureuser/.demoji/codes.json ...
... OK


In [6]:
tweets_df = pd.read_json('../input/iphone-12-tweets-fr.json', lines=True)
# garder que les colonnes importantes
cols = ['date', 'content', 'lang']
tweets_df = tweets_df[cols]
# pour etre sur que tout les tweets sont en français
tweets_df = tweets_df[tweets_df['lang'] == 'fr']
tweets_df['lang'].unique()
# supprimer les tweets dupliqués, 1K ont été supprimés

tweets_df = tweets_df.sort_values("content") 
  
# dropping ALL duplicte values 
tweets_df = tweets_df.drop_duplicates(subset ="content", keep = 'first')

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
# selectionner que les tweets qui répondent au requetes de l'utilisateur

data = tweets_df['content']

keywords = ['stockage', 'écran', 'ecran', 'autonomie', 'réseau', 'reseau', 'alimentation',
           'appareil', 'appareil photo', 'processeur', 'multimédia', 'multimedia', 'résolution', 'resolution',
           'batterie', 'os', 'pièces', 'pieces' , 'couleur',
           'communication', 'sans fil', 'synchronisation', 'coloris', 'poids', 'système', 'systeme',
           'dimensions']

data = data[data.str.contains('|'.join(keywords), case=False)]

ads_words = [
    '#concours', '#jeuconcours', '#giveaway',
    '#gagne' ,'#gangner', '#promo', '#promotion', '#publicité',
    '#contest', '#ad', '#pub', '#réduction', '#vote', 'votez', 'vote']

data = data[~data.str.contains('|'.join(ads_words), case=False)]

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
df = pd.DataFrame(data)
df.columns = ['original_tweets']

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    return demoji.replace(text, '').strip()

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub(r'(?:\@|https?\://)\S+', '', text)
    return text

# remplacement des abréviations

with open('../input/abrivot_fr.json', encoding='utf-8') as f:
    abrivot = json.load(f)
    
def replace_abrivot(text):
    '''
    input: string
    output: string
    '''
    words = text.lower().split()
    text_out = [abrivot[word] if word in abrivot else word for word in words]
    return ' '.join(text_out)

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
# Apply `call_emoji_free` which calls the function to remove all emoji's
df['emoji_free_tweets'] = df['original_tweets'].apply(give_emoji_free_text)

#Create a new column with url free tweets
df['url_free_tweets'] = df['emoji_free_tweets'].apply(url_free_text)

df['abrivots_free_tweets'] = df['url_free_tweets'].apply(replace_abrivot)

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
# Load spacy
# Make sure to restart the runtime after running installations and libraries tab
nlp = spacy.load('fr_core_news_sm')

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# mettre à jour la liste d'aprés le fichies des mots vides
with open('../input/fr_stopwords.txt', encoding='utf-8') as f:
    fr_stopwords = f.read().splitlines()
    
# Custom stopwords
custom_stopwords = ['\n','\n\n', '&amp;', ' ', '.', '-','$', '@']

# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)

# ALL_STOP_WORDS = spacy + gensim + wordcloud
ALL_STOP_WORDS = STOP_WORDS.union(fr_stopwords)

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

tokens = []

for doc in tokenizer.pipe(df['abrivots_free_tweets'], batch_size=500):
    doc_tokens = []    
    for token in doc: 
        if token.text.lower() not in ALL_STOP_WORDS:
            doc_tokens.append(token.text.lower())   
    tokens.append(doc_tokens)

# Makes tokens column
df['tokens'] = tokens

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
# Make tokens a string again
df['tokens_back_to_text'] = [' '.join(map(str, l)) for l in df['tokens']]

def get_lemmas(text):
    '''Used to lemmatize the processed tweets'''
    lemmas = []
    
    doc = nlp(text)
    
    # Something goes here :P
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON'):
            lemmas.append(token.lemma_)
    
    return lemmas

df['lemmas'] = df['tokens_back_to_text'].apply(get_lemmas)

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
# Make lemmas a string again
df['lemmas_back_to_text'] = [' '.join(map(str, l)) for l in df['lemmas']]

# Tokenizer function
def tokenize(text):
    """
    Parses a string into a list of semantic units (words)
    Args:
        text (str): The string that the function will tokenize.
    Returns:
        list: tokens parsed out
    """
    # Removing url's
    pattern = r"http\S+"
    
    tokens = re.sub(pattern, "", text) # https://www.youtube.com/watch?v=O2onA4r5UaY
    tokens = re.sub('[^a-zA-Z 0-9]', '', text)
    tokens = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Remove punctuation
    tokens = re.sub('\w*\d\w*', '', text) # Remove words containing numbers
    tokens = re.sub('@*!*\$*', '', text) # Remove @ ! $
    tokens = tokens.strip(',') # TESTING THIS LINE
    tokens = tokens.strip('?') # TESTING THIS LINE
    tokens = tokens.strip('!') # TESTING THIS LINE
    tokens = tokens.strip("'") # TESTING THIS LINE
    tokens = tokens.strip(".") # TESTING THIS LINE

    tokens = tokens.lower().split() # Make text lowercase and split it
    
    return tokens

# Apply tokenizer
df['lemma_tokens'] = df['lemmas_back_to_text'].apply(tokenize)

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:19: DeprecationWarning: invalid escape sequence \w
<>:20: DeprecationWarning: invalid escape sequence \$
<>:19: DeprecationWarning: invalid escape sequence \w
<>:20: DeprecationWarning: invalid escape sequence \$
<ipython-input-15-6a54c065e71a>:19: DeprecationWarning: invalid escape sequence \w
  tokens = re.sub('\w*\d\w*', '', text) # Remove words containing numbers
<ipython-input-15-6a54c065e71a>:20: DeprecationWarning: invalid escape sequence \$
  tokens = re.sub('@*!*\$*', '', text) # Remove @ ! $


In [16]:
df

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,original_tweets,emoji_free_tweets,url_free_tweets,abrivots_free_tweets,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text,lemma_tokens
19381,"""\n \n Actualité : L...","""\n \n Actualité : L...","""\n \n Actualité : L...",""" actualité : le format apple proraw disponibl...","["", actualité, :, format, apple, proraw, dispo...",""" actualité : format apple proraw disponible i...","[actualité, format, apple, proraw, disponible,...",actualité format apple proraw disponible iphon...,"[actualité, format, apple, proraw, disponible,..."
6029,""" Apple est une marque de snobinards "" vient d...",""" Apple est une marque de snobinards "" vient d...",""" Apple est une marque de snobinards "" vient d...",""" apple est une marque de snobinards "" vient d...","["", apple, marque, snobinards, "", vient, comma...",""" apple marque snobinards "" vient commander ip...","[apple, marqu, snobinard, venir, commander, ip...",apple marqu snobinard venir commander iphon 12...,"[apple, marqu, snobinard, venir, commander, ip..."
2833,"""Anker lance sa propre batterie externe MagSaf...","""Anker lance sa propre batterie externe MagSaf...","""Anker lance sa propre batterie externe MagSaf...","""anker lance sa propre batterie externe magsaf...","[""anker, lance, propre, batterie, externe, mag...","""anker lance propre batterie externe magsafe i...","[anker, lance, propre, batterie, externe, mags...",anker lance propre batterie externe magsafe ip...,"[anker, lance, propre, batterie, externe, mags..."
32661,"""Apple : certains iPhone 12 mini souffrent d'u...","""Apple : certains iPhone 12 mini souffrent d'u...","""Apple : certains iPhone 12 mini souffrent d'u...","""apple : certains iphone 12 mini souffrent d'u...","[""apple, :, iphone, 12, mini, souffrent, d'un,...","""apple : iphone 12 mini souffrent d'un problèm...","[apple, iphone, 12, mini, souffrir, problème, ...",apple iphone 12 mini souffrir problème réactiv...,"[apple, iphone, 12, mini, souffrir, problème, ..."
30812,"""Apple iPhone 12 : certaines unités ont des pr...","""Apple iPhone 12 : certaines unités ont des pr...","""Apple iPhone 12 : certaines unités ont des pr...","""apple iphone 12 : certaines unités ont des pr...","[""apple, iphone, 12, :, unités, problèmes, d’é...","""apple iphone 12 : unités problèmes d’écran""","[apple, iphone, 12, unité, problème, écran]",apple iphone 12 unité problème écran,"[apple, iphone, 12, unité, problème, écran]"
...,...,...,...,...,...,...,...,...,...
37310,🤩iPhone 12🤩\n\nOn craque pour le rouge de l'iP...,iPhone 12\n\nOn craque pour le rouge de l'iPho...,iPhone 12\n\nOn craque pour le rouge de l'iPho...,iphone 12 on craque pour le rouge de l'iphone ...,"[iphone, 12, craque, rouge, l'iphone, 12,, ?, ...","iphone 12 craque rouge l'iphone 12, ? venez dé...","[iphone, 12, craque, rouge, iphone, 12, venez,...",iphone 12 craque rouge iphone 12 venez découvr...,"[iphone, 12, craque, rouge, iphone, 12, venez,..."
53785,🤪 Bon avec tout ce buzz autour de Macron et le...,Bon avec tout ce buzz autour de Macron et le #...,Bon avec tout ce buzz autour de Macron et le #...,bon avec tout ce buzz autour de macron et le #...,"[buzz, autour, macron, #covid19, ,, sortie, l’...","buzz autour macron #covid19 , sortie l’iphone ...","[buzz, autour, macron, covid19, sortie, iphon,...",buzz autour macron covid19 sortie iphon 12 min...,"[buzz, autour, macron, covid19, sortie, iphon,..."
64693,🦄 iPhone 12 : une autonomie sensiblement plus ...,iPhone 12 : une autonomie sensiblement plus im...,iPhone 12 : une autonomie sensiblement plus im...,iphone 12 : une autonomie sensiblement plus im...,"[iphone, 12, :, autonomie, sensiblement, impor...",iphone 12 : autonomie sensiblement importante ...,"[iphone, 12, autonomie, sensiblement, importan...",iphone 12 autonomie sensiblement important media,"[iphone, 12, autonomie, sensiblement, importan..."
32239,🧑🏻‍💻Tweet 👇🏻\nRemarque(s) sur les batteries de...,Tweet \nRemarque(s) sur les batteries des nouv...,Tweet \nRemarque(s) sur les batteries d

# Topic Modeling

In [17]:
# Create a id2word dictionary
id2word = Dictionary(df['lemma_tokens'])
print(len(id2word))

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


13917


In [18]:
# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)
print(len(id2word))

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


6370


In [19]:
# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in df['lemma_tokens']]

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
# Instantiating a Base LDA model 
base_model = LdaMulticore(corpus=corpus, num_topics=7, id2word=id2word, workers=12, passes=5)


In [24]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]

In [25]:
# Create Topics
topics = [' '.join(t[0:10]) for t in words]

In [26]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
12 iphone mini iphon écran apple pro euro 5 11

------ Topic 1 ------
12 iphone coque accessoire test protection vitre jcsataner etui support

------ Topic 2 ------
12 iphon iphone pro écran max plus photo acheter euro

------ Topic 3 ------
12 iphon iphone batterie photo gros prendre apple acheter magsafe

------ Topic 4 ------
12 iphon iphone pro apple écran max couleur coque mini

------ Topic 5 ------
12 iphone pro iphon max 5 gramme apple batterie io

------ Topic 6 ------
12 pro iphon iphone photo max couleur bleu apple appareil



In [27]:
# Compute Perplexity
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=df['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -6.659952673761381

Coherence Score:  0.31316992288064555


In [32]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(base_model, corpus, id2word)

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.021125  0.009664       1        1  21.664731
0     -0.048294  0.018428       2        1  16.813131
2     -0.047232  0.015565       3        1  15.852732
3     -0.078868  0.077818       4        1  12.574633
6     -0.019213 -0.121290       5        1  12.406765
4      0.022982 -0.033606       6        1  11.153856
1      0.191750  0.033421       7        1   9.534151, topic_info=           Term         Freq        Total Category  logprob  loglift
395       coque   794.000000   794.000000  Default  30.0000  30.0000
679  accessoire   672.000000   672.000000  Default  29.0000  29.0000
11          pro  4508.000000  4508.000000  Default  28.0000  28.0000
9         iphon  5444.000000  5444.000000  Default  27.0000  27.0000
94        photo  1324.000000  1324.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
60         euro    66.452027   983.610253   Topic7  -5.2968  -0.3445
156     acheter    46.081406   608.765385   Topic7  -5.6629  -0.2307
18    téléphone    37.608562   381.322381   Topic7  -5.8661   0.0339
593     vouloir    30.422897   220.039522   Topic7  -6.0781   0.3717
8            io    34.015566   851.707067   Topic7  -5.9665  -0.8701

[578 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
6051      2  0.850196        017
1901      1  0.105403       1000
1901      2  0.118578       1000
1901      3  0.592891       1000
1901      4  0.065877       1000
...     ...       ...        ...
2023      2  0.079113  évolution
5767      1  0.861802      évoqu
5767      2  0.123115      évoqu
6348      1  0.122570         ️️
6348      7  0.857987         ️️

[1569 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 3, 4, 7, 5, 2])

# Hypermarameter Tuning

In [21]:
vectorizer = CountVectorizer()
data_vectorized = vectorizer.fit_transform(df['lemmas_back_to_text'])

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:


# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score='raise',
             estimator=LatentDirichletAllocation(batch_size=128, 
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1, 
                                                 learning_decay=0.7, 
                                                 learning_method=None,
                                                 learning_offset=10.0, 
                                                 max_doc_update_iter=100, 
                                                 max_iter=10,
                                                 mean_change_tol=0.001, 
                                                 n_components=10, 
                                                 n_jobs=1,
                                                 perp_tol=0.1, 
                                                 random_state=None,
                                                 topic_word_prior=None, 
                                                 total_samples=1000000.0, 
                                                 verbose=0),
             iid=True, n_jobs=1,
             param_grid={'n_topics': [10, 15, 20, 30], 
                         'learning_decay': [0.5, 0.7, 0.9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
             scoring=None, verbose=0)

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


GridSearchCV(error_score='raise',
             estimator=LatentDirichletAllocation(learning_method=None,
                                                 n_jobs=1),
             iid=True, n_jobs=1,
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_topics': [10, 15, 20, 30]},
             return_train_score='warn')

In [23]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Best Model's Params:  {'learning_decay': 0.7, 'n_components': 10}
Best Log Likelihood Score:  -273067.72970150394
Model Perplexity:  1066.0463717410612


In [24]:
#Defining a function to loop over number of topics to be used to find an 
#optimal number of tipics
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the 
    LDA model with respective number of topics
    """
    coherence_values_topic = []
    model_list_topic = []
    for num_topics in range(start, limit, step):
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list_topic.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values_topic.append(coherencemodel.get_coherence())

    return model_list_topic, coherence_values_topic

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
# Can take a long time to run.
model_list_topic, coherence_values_topic = compute_coherence_values(dictionary=id2word,
                                                        corpus=corpus,
                                                        texts=df['lemma_tokens'],
                                                        start=2, limit=200, step=6)

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
model_5_2 = LdaMulticore(corpus=corpus,
                       id2word=id2word,
                       num_topics=68,
                       random_state=42,
                       chunksize=2000,
                       passes=25,
                       decay=0.5,
                       iterations=70)

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
# Compute Perplexity
# a measure of how good the model is. lower the better
base_perplexity = model_5_2.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=model_5_2, texts=df['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -6.7486625605392865

Coherence Score:  0.3349241970931369


In [28]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model_5_2, corpus, id2word)

/anaconda/envs/pfe-env/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
32    -0.137086 -0.078838       1        1  3.651492
0     -0.004718 -0.346031       2        1  3.068474
22    -0.154745 -0.289846       3        1  2.972027
40     0.093366 -0.107188       4        1  2.600292
10    -0.118789  0.010569       5        1  2.299950
...         ...       ...     ...      ...       ...
28     0.077107 -0.010092      64        1  0.937713
2     -0.008031  0.089876      65        1  0.914689
7      0.059093  0.030134      66        1  0.912402
41     0.051696  0.007396      67        1  0.908078
56     0.041653 -0.038086      68        1  0.872895

[68 rows x 5 columns], topic_info=         Term         Freq        Total Category  logprob  loglift
21   batterie  1040.000000  1040.000000  Default  30.0000  30.0000
94      photo  1238.000000  1238.000000  Default  29.0000  29.0000
82          5  1012.000000  1012.000000  Default  28.0000  28.0000
60       euro  1000.000000  1000.000000  Default  27.0000  27.0000
85     gramme   836.000000   836.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
27       mini    15.876260  1424.394019  Topic68  -4.3376   0.2444
9       iphon    19.880021  5312.265968  Topic68  -4.1128  -0.8469
11        pro    12.171923  4566.480078  Topic68  -4.6033  -1.1863
82          5     8.781091  1012.808808  Topic68  -4.9299  -0.0068
401       air     7.818777   124.801654  Topic68  -5.0459   1.9709

[3924 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
427      17  0.823466       +221
427      47  0.126687       +221
5157     22  0.827253       +226
5804     13  0.673797        +87
5029     53  0.788586   +8€/mois
...     ...       ...        ...
434      49  0.831914          ️
434      53  0.113443          ️
6343     26  0.706561  ️nouvelle
6347     38  0.628690         ️️
6347     62  0.209563         ️️

[10057 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[33, 1, 23, 41, 11, 37, 30, 10, 49, 28, 32, 40, 46, 17, 68, 64, 5, 65, 62, 39, 66, 24, 58, 51, 13, 52, 47, 22, 27, 12, 21, 60, 18, 6, 35, 36, 63, 67, 16, 54, 9, 61, 50, 43, 2, 56, 31, 45, 44, 38, 48, 15, 4, 26, 55, 7, 53, 19, 34, 59, 25, 14, 20, 29, 3, 8, 42, 57])